In [1]:
import os
import mlflow
from mlflow import MlflowClient
import torch
import torchmetrics
import random
import numpy as np
from tqdm import tqdm
import sys

sys.path.append('./')
from src.utils.data_utils import read_dataset, read_dataset_config
from src.trainers.AETrainer import AEFCNTrainer

DATA_PATH = './data'
FILE_NAME_TRAIN = 'BP_safety_network_master_NN_train.csv'
FILE_NAME_TEST = 'BP_safety_network_master_NN_test.csv'
FILE_NAME_CONF = 'BP_safety_network_master_NN_config.csv'
AE_RUN_ID = 'e5a5b25fc25f4435bd9177664c21ff9a'
RANDOM_SEED = 42

mlflow.set_tracking_uri('http://127.0.0.1:8080')

In [2]:
experiment = mlflow.set_experiment('02_FCN')

In [3]:
with mlflow.start_run(log_system_metrics=True) as run:
    # Seed random generators to ensure deterministic experiments
    random.seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)
    torch.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Define PyTorch device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Read and log train, validation, test datasets and dataset config file
    X_train, y_train, _ = read_dataset(DATA_PATH, FILE_NAME_TRAIN, targets='rs_crashes_2324', device=device)
    X_test, y_test, non_accident_dim = read_dataset(DATA_PATH, FILE_NAME_TEST, targets='rs_crashes_2324', device=device)
    df_conf = read_dataset_config(DATA_PATH, FILE_NAME_CONF)
    
    # Load trained autoencoder from MLflow
    autoencoder = mlflow.pytorch.load_model(f'runs:/{AE_RUN_ID}/{MlflowClient().list_artifacts(AE_RUN_ID, "models")[0].path}')
    
    # Specify and log training parameters
    params = {
        'autoencoder': autoencoder,
        'inp_dim': X_train.shape[1] - non_accident_dim + autoencoder.enc_dim,
        'learning_rate': 1e-2
    }
    mlflow.log_params(params.copy())

    # Define, train and evaluate model
    trainer = AEFCNTrainer(**params)
    trainer.train(X_train, y_train, X_test, y_test, epochs=200, min_max_norms=(df_conf['rs_crashes_min'][0], df_conf['rs_crashes_max'][0]))
    trainer.evaluate(X_test, y_test, 'test', min_max_norms=(df_conf['rs_crashes_min'][0], df_conf['rs_crashes_max'][0]))
    trainer.evaluate(X_train, y_train, 'train', torch.nn.L1Loss(), 'mae', min_max_norms=(df_conf['rs_crashes_min'][0], df_conf['rs_crashes_max'][0]))
    trainer.evaluate(X_test, y_test, 'test', torch.nn.L1Loss(), 'mae', min_max_norms=(df_conf['rs_crashes_min'][0], df_conf['rs_crashes_max'][0]))
    trainer.evaluate(X_train, y_train, 'train', torchmetrics.regression.MeanAbsolutePercentageError(), 'mape',
                     min_max_norms=(df_conf['rs_crashes_min'][0], df_conf['rs_crashes_max'][0]), proc_func=lambda x: x+1)
    trainer.evaluate(X_test, y_test, 'test', torchmetrics.regression.MeanAbsolutePercentageError(), 'mape',
                     min_max_norms=(df_conf['rs_crashes_min'][0], df_conf['rs_crashes_max'][0]), proc_func=lambda x: x+1)

2025/10/02 12:50:57 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: Failed to initialize NVML, skip logging GPU metrics: NVML Shared Library Not Found.
2025/10/02 12:50:57 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
C:\Users\galig\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:12<00:00, 15.89it/s]
2025/10/02 12:51:28 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/10/02 12:51:28 INFO mlflow.system_metrics.system_metrics_monitor: Successfully termi

🏃 View run mercurial-quail-875 at: http://127.0.0.1:8080/#/experiments/196378654275874061/runs/864acd819e184220b6ca130a66e2e061
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/196378654275874061
